In [ ]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation file
path_to_data = r"C:\Users\phani\OneDrive\Trainings\IP Training AI\AI and Deep Learning using TensorFLow\NLP\spa-eng\spa.txt"

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8')
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs = pairs[1000:20000]

In [ ]:
translation_file = open(path_to_data,"r", encoding='utf-8')
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs = pairs[1000:20000]

In [ ]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))

    return clean_sentence

def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:
sample = ["Hello How are you", "Hello Hi"]
english_sentences = [clean_sentence(sent) for sent in sample]
eng_text_tokenized, eng_text_tokenizer = tokenize(sample)
print(eng_text_tokenized)

[[1, 2, 3, 4], [1, 5]]


In [ ]:

# Clean sentences
english_sentences = [clean_sentence(pair[0]) for pair in pairs]
spanish_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
spa_text_tokenized, spa_text_tokenizer = tokenize(spanish_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length spanish sentence: {}'.format(len(max(spa_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
spanish_vocab = len(spa_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of {} unique words".format(spanish_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length spanish sentence: 9
Maximum length english sentence: 5
Spanish vocabulary is of 7265 unique words
English vocabulary is of 3722 unique words


In [ ]:
max_spanish_len = int(len(max(spa_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

spa_pad_sentence = pad_sequences(spa_text_tokenized, max_spanish_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# Reshape data
spa_pad_sentence = spa_pad_sentence.reshape(*spa_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [ ]:
#

model = Sequential()
model.add()
model.add()

In [ ]:
### Model (Functional API)
input_sequence = Input(shape=(max_spanish_len,)) # Sequence of integers
embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)

In [ ]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding_2 (Embedding)     (None, 9, 128)            929920    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVecto  (None, 5, 64)             0         
 r)                                                              
                                                                 
 lstm_2 (LSTM)               (None, 5, 64)             33024     
                                                                 
 time_distributed (TimeDist  (None, 5, 3722)           241930    
 ributed)                                                    

In [ ]:
model_results = enc_dec_model.fit(spa_pad_sentence, eng_pad_sentence, batch_size=30, epochs=100)

Epoch 1/100
634/634 [==============================] - 20s 25ms/step - loss: 4.5318 - accuracy: 0.3599
Epoch 2/100
634/634 [==============================] - 15s 24ms/step - loss: 3.9573 - accuracy: 0.3931
Epoch 3/100
634/634 [==============================] - 15s 24ms/step - loss: 3.7564 - accuracy: 0.4036
Epoch 4/100
634/634 [==============================] - 15s 24ms/step - loss: 3.5516 - accuracy: 0.4207
Epoch 5/100
634/634 [==============================] - 16s 25ms/step - loss: 3.3395 - accuracy: 0.4435
Epoch 6/100
634/634 [==============================] - 17s 28ms/step - loss: 3.1206 - accuracy: 0.4720
Epoch 7/100
634/634 [==============================] - 16s 25ms/step - loss: 2.9385 - accuracy: 0.4934
Epoch 8/100
634/634 [==============================] - 16s 25ms/step - loss: 2.7829 - accuracy: 0.5114
Epoch 9/100
634/634 [==============================] - 16s 25ms/step - loss: 2.6449 - accuracy: 0.5273
Epoch 10/100
634/634 [==============================] - 16s 25ms/step - l

In [ ]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 14
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(spanish_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(spa_pad_sentence[index:index+1])[0], eng_text_tokenizer))

The english sentence is: seize him
The spanish sentence is: prendedlo
The predicted sentence is :
1/1 [==============================] - 0s 37ms/step
seize him <empty> <empty> <empty>
